#### Pourquoi il est possible de déduire automatiquement le type d’hôtel

Exemple typique :
##### Hôtel urbain/business
    Très fort en semaine
    Baisse brutale le week-end
    Été = creux (Juillet–Août)
    PM élevé, lead-time court
    Très sensible aux événements/congrès

#### Hôtel loisirs bord de mer
    Forte saison Juin → Septembre
    Gros creux hors saison
    Week-ends = montée
    Lead-time long
    Prix très saisonniers

#### Hôtel montagne / ski
    Pics Décembre–Janvier et Février
    Très faible le reste de l'année
    Très dépendant du calendrier scolaire

#### Hôtel autoroute / économique
    Forte stabilité annuelle
    Week-end souvent plus faible
    Peu de saisonnalité
    Prix stable

Ces signaux permettent un clustering automatique très précis.


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# -------------------------------------------------------------
# 1. Chargement des données
# -------------------------------------------------------------
hotCode='F4L'   

df = pd.read_csv('../data/'+hotCode+'/Indicateurs.csv', sep=';')
df.fillna(0, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
df['Weekday'] = df['Date'].dt.weekday


# ----------------------------------------------
# 1. Feature engineering : profil unique de l'hôtel
# ----------------------------------------------

TO_monthly = df.groupby('Month')['To'].mean()
PM_monthly = df.groupby('Month')['Pm'].mean()

profile = {
    # Semaine vs weekend
    "TO_weekday": df[df.Weekday < 5]['To'].mean(),
    "TO_weekend": df[df.Weekday >= 5]['To'].mean(),
    "weekend_ratio": df[df.Weekday >= 5]['To'].mean() / df[df.Weekday < 5]['To'].mean(),

    # Pics été
    "summer_peak": TO_monthly.loc[6:8].mean(),   # Juin → Août

    # Pics hiver
    "winter_peak": pd.concat([TO_monthly.loc[12:12], TO_monthly.loc[1:3]]).mean(),

    # Amplitude saisonnière
    "seasonality_amplitude": TO_monthly.max() - TO_monthly.min(),

    # Variation prix
    "pm_seasonality": PM_monthly.max() - PM_monthly.min(),

    # Lead-time moyen
    "avg_lead_time": df['Ant'].mean(),

    # Stabilité du pickup
    "pickup_speed": df['Sel'].mean() if "Sel" in df.columns else None,
}

# Petit utilitaire
def pct(x):
    return round(float(x)*100,2)

# ----------------------------------------------
# 2. Classifieur RULE-BASED (expert RMS)
# ----------------------------------------------

def detect_hotel_type(p):

    # ========== 1. HÔTEL MER / LOISIRS ==========
    if (
        p["summer_peak"] > p["winter_peak"] * 1.4  # été beaucoup plus fort
        and p["seasonality_amplitude"] > 0.25      # forte saisonnalité
        and p["weekend_ratio"] > 1.0              # weekend > semaine
    ):
        return "Hôtel Mer / Loisirs"

    # ========== 2. HÔTEL MONTAGNE / SKI ==========
    if (
        p["winter_peak"] > p["summer_peak"] * 1.4  # hiver >> été
        and p["seasonality_amplitude"] > 0.25
    ):
        return "Hôtel Montagne / Ski"

    # ========== 3. URBAIN / BUSINESS ==========
    if (
        p["TO_weekday"] > p["TO_weekend"] * 1.25   # semaine >>> weekend
        and p["summer_peak"] < p["winter_peak"]    # été creux
        and p["seasonality_amplitude"] < 0.20      # faible saisonnalité
    ):
        return "Hôtel Urbain / Business"

    # ========== 4. ROUTE / ÉCONOMIQUE ==========
    if (
        p["seasonality_amplitude"] < 0.10           # très peu de saisonnalité
        and p["weekend_ratio"] < 0.9                # weekend un peu plus faible
        and p["pm_seasonality"] < 15                # prix très stables
    ):
        return "Hôtel Routier / Économique"

    # ========== 5. LOISIRS GÉNÉRAL ==========
    if (
        p["weekend_ratio"] > 1.1 or
        p["seasonality_amplitude"] > 0.15
    ):
        return "Hôtel Loisirs Général"

    # ========== 6. PAR DÉFAUT ==========
    return "Hôtel Indéterminé (profil mixte)"


# ----------------------------------------------
# 3. Exécution
# ----------------------------------------------

hotel_type = detect_hotel_type(profile)

print("===== Profil Hôtel détecté =====")
print("Type :", hotel_type)
print("\nDétails utilisés pour l'analyse :")
for k,v in profile.items():
    print(f" - {k}: {v}")



===== Profil Hôtel détecté =====
Type : Hôtel Loisirs Général

Détails utilisés pour l'analyse :
 - TO_weekday: 0.0705458091061269
 - TO_weekend: 0.07167140289409589
 - weekend_ratio: 1.0159555018537767
 - summer_peak: 0.12760618993070438
 - winter_peak: 0.010837524529882042
 - seasonality_amplitude: 0.15435608913142337
 - pm_seasonality: 221.61450397877985
 - avg_lead_time: 27.775488012869687
 - pickup_speed: 1.3480699193410244
